In [1]:
import os
import cv2
import sys
import torch
import numpy as np
from math import sin, cos

from tqdm.auto import tqdm
from torch.utils.data import Dataset
from typing import Union, Tuple, List, Dict, Any

from utils.engine_utils import tprint
from utils.geometry_ops import extract_corners_from_bboxes_3d, points_cam2img
from utils.data_classes import KITTICalibration
# from kitti_eval.rotate_iou import rotate_iou_gpu_eval

def np_computeBox3D(label, P):
    w = label[0]
    h = label[1]
    l = label[2]
    # w = label[1]
    # h = label[2]
    # l = label[0]
    x = label[3]
    y = label[4]
    z = label[5]
    ry = label[6]

    R = np.array([[+cos(ry), 0, +sin(ry)],
                  [0, 1, 0],
                  [-sin(ry), 0, +cos(ry)]])

    x_corners = [0, l, l, l, l, 0, 0, 0]  # -l/2
    y_corners = [0, 0, h, h, 0, 0, h, h]  # -h
    z_corners = [0, 0, 0, w, w, w, w, 0]  # -w/2

    x_corners = [i - l / 2 for i in x_corners]
    y_corners = [i - h for i in y_corners]
    z_corners = [i - w / 2 for i in z_corners]

    corners_3D = np.array([x_corners, y_corners, z_corners])

    corners_3D = R.dot(corners_3D)

    corners_3D += np.array([x, y, z]).reshape((3, 1))

    corners_3D_1 = np.vstack((corners_3D, np.ones((corners_3D.shape[-1]))))

    corners_2D = P.dot(corners_3D_1)

    corners_2D = corners_2D / corners_2D[2]

    corners_2D = corners_2D[:2]
    ver_coor = corners_2D[:, [4, 3, 2, 1, 5, 6, 7, 0]]
    return corners_2D, corners_3D, ver_coor

def plot_bboxes_3d(image_file: str, calib_file: str, pred_file, gt_file, save_path: str = None) -> Union[None, np.ndarray]:

    image_arr = cv2.imread(image_file)
    # print(self.image_files[idx])
    image = cv2.cvtColor(image_arr, code=cv2.COLOR_BGR2RGB)

    # Load Calib
    calib = KITTICalibration(calib_file)
    intrinsic_mat = calib.P2                        # (3, 4)

    # Load 3D Predicted Boxes
    with open(pred_file) as f:
        lines = f.readlines()
    bboxes = []
    for line in lines:
        bbox = line.strip().split(' ')[8:15]
        bbox_3d = bbox[3:6] + bbox[0:3]
        bbox_3d.append(bbox[6])
        bboxes.append(bbox)

    with open(gt_file) as f:
        lines = f.readlines()
    gt_bboxes = []
    for line in lines:
        bbox = line.strip().split(' ')[8:15]
        bbox_3d = bbox[3:6] + bbox[0:3]
        bbox_3d.append(bbox[6])
        gt_bboxes.append(bbox)

    bboxes = np.array(bboxes, dtype=np.float32)
    pred_bboxes_3d = torch.tensor(bboxes)

    gt_bboxes = np.array(gt_bboxes, dtype=np.float32)
    gt_bboxes_3d = torch.tensor(gt_bboxes)

    if len(pred_bboxes_3d) > 0:
        # Draw 3D Boxes
        line_indices = ((0, 1), (0, 3), (0, 4), (1, 2), (1, 5), (3, 2), (3, 7),
                        (4, 5), (4, 7), (2, 6), (5, 6), (6, 7))

        for bbox_3d in pred_bboxes_3d:
            _, _, ver_coor = np_computeBox3D(bbox_3d, intrinsic_mat)
            ver_coor = np.transpose(ver_coor)
            # print(ver_coor)

            proj_corners = np.array(ver_coor, dtype=int)

            color = (255, 0, 0)
            for start, end in line_indices:
                image = cv2.line(image,
                                (proj_corners[start, 0], proj_corners[start, 1]),
                                (proj_corners[end, 0], proj_corners[end, 1]),
                                color,
                                thickness=1,
                                lineType=cv2.LINE_AA)

    if len(gt_bboxes_3d) > 0:
        # Draw 3D Boxes
        line_indices = ((0, 1), (0, 3), (0, 4), (1, 2), (1, 5), (3, 2), (3, 7),
                        (4, 5), (4, 7), (2, 6), (5, 6), (6, 7))

        for bbox_3d in gt_bboxes_3d:
            _, _, ver_coor = np_computeBox3D(bbox_3d, intrinsic_mat)
            ver_coor = np.transpose(ver_coor)
            # print(ver_coor)

            proj_corners = np.array(ver_coor, dtype=int)

            color = (0, 255, 0)
            for start, end in line_indices:
                image = cv2.line(image,
                                (proj_corners[start, 0], proj_corners[start, 1]),
                                (proj_corners[end, 0], proj_corners[end, 1]),
                                color,
                                thickness=1,
                                lineType=cv2.LINE_AA)

    if save_path is not None:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, image)
    else:
        return image

In [3]:
def plot_bboxes_bev(image_file: str, calib_file: str, pred_file, gt_file, save_path: str = None) -> Union[None, np.ndarray]:

    image_arr = cv2.imread(image_file)
    # print(self.image_files[idx])
    image = cv2.cvtColor(image_arr, code=cv2.COLOR_BGR2RGB)

    # Load Calib
    calib = KITTICalibration(calib_file)
    intrinsic_mat = calib.P2                        # (3, 4)

    # Load 3D Predicted Boxes
    with open(pred_file) as f:
        lines = f.readlines()
    bboxes = []
    for line in lines:
        bbox = line.strip().split(' ')[8:15]
        bbox_3d = bbox[3:6] + bbox[0:3]
        bbox_3d.append(bbox[6])
        bboxes.append(bbox)

    with open(gt_file) as f:
        lines = f.readlines()
    gt_bboxes = []
    for line in lines:
        bbox = line.strip().split(' ')[8:15]
        bbox_3d = bbox[3:6] + bbox[0:3]
        bbox_3d.append(bbox[6])
        gt_bboxes.append(bbox)

    bboxes = np.array(bboxes, dtype=np.float32)
    pred_bboxes_3d = torch.tensor(bboxes)

    gt_bboxes = np.array(gt_bboxes, dtype=np.float32)
    gt_bboxes_3d = torch.tensor(gt_bboxes)

    MAX_DIST = 60
    SCALE = 10

    # Create BEV Space
    R = (MAX_DIST * SCALE)
    space = np.zeros((R * 2, R * 2, 3), dtype=np.uint8)

    for theta in np.linspace(0, np.pi, 7):
        space = cv2.line(space,
                        pt1=(int(R - R * np.cos(theta)), int(R - R * np.sin(theta))),
                        pt2=(R, R),
                        color=(255, 255, 255),
                        thickness=2,
                        lineType=cv2.LINE_AA)

    for radius in np.linspace(0, R, 5):
        if radius == 0:
            continue

        space = cv2.circle(space,
                           center=(R, R),
                           radius=int(radius),
                           color=(255, 255, 255),
                           thickness=2,
                           lineType=cv2.LINE_AA)
    space = space[:R, :, :]

    if len(pred_bboxes_3d) > 0:
        pred_bev = pred_bboxes_3d[:, [3, 5, 2, 1, 6]]
        pred_bev[:, :-1] *= SCALE
        pred_bev[:, 1] *= (-1)
        pred_bev[:, :2] += R
        for idx, bev in enumerate(pred_bev):
            bev = tuple(bev.numpy())
            box = cv2.boxPoints((bev[:2], bev[2:4], (bev[4] * 180 / np.pi)))
            box = np.int0(box)
            color = (255, 0, 0)
            space = cv2.drawContours(space, [box], -1, color, thickness=1, lineType=cv2.LINE_AA)

    if len(gt_bboxes_3d) > 0:
        pred_bev = gt_bboxes_3d[:, [3, 5, 2, 1, 6]]
        pred_bev[:, :-1] *= SCALE
        pred_bev[:, 1] *= (-1)
        pred_bev[:, :2] += R
        for idx, bev in enumerate(pred_bev):
            bev = tuple(bev.numpy())
            box = cv2.boxPoints((bev[:2], bev[2:4], (bev[4] * 180 / np.pi)))
            box = np.int0(box)
            color = (0, 255, 0)
            space = cv2.drawContours(space, [box], -1, color, thickness=1, lineType=cv2.LINE_AA)

    if save_path is not None:
        space = cv2.cvtColor(space, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, space)
    else:
        return image

In [45]:
import engine.kitti_eval.eval
from engine.kitti_eval.eval import d3_box_overlap
from kitti_eval.rotate_iou import rotate_iou_gpu_eval

box1 = np.array([[-12.66, 1.13, 38.44, 4.29, 1.68, 1.67, 1.73]])
# box2 = np.array([[1.53, 3.46,  -2.33, 28.83, -1.24]])
box2 = np.array([[-11.122984886169434, 1.1064577102661133, 33.893280029296875, 4.0518574714660645, 1.4804763793945312, 1.6470861434936523, 1.7667932510375977]])
# print(rotate_iou_gpu_eval(box1, box2, 2))
print(d3_box_overlap(box1, box2, -1))

box1 = np.array([[0, 0, 0, 1, 1, 1, 0]])
box2 = np.array([[0, 0, 0, 1, 1, 1, 0.785]])
print(d3_box_overlap(box1, box2, -1))

[[-12.66  38.44   4.29   1.67   1.73]]
[[0.]]
[[0 0 1 1 0]]
[[0.70710677]]


In [4]:
import os
from tqdm import tqdm
directory = 'Kitti/iou'

preds_iou = []
preds_l1 = []
gts = []
images = []
calibs = []
saves_3d_iou = []
saves_bev_iou = []
saves_3d_l1 = []
saves_bev_l1 = []

for filename in os.listdir(directory):
    iou = os.path.join(directory, filename)
    if os.path.isfile(iou):
        # print(iou)
        preds_iou.append(iou)
        preds_l1.append(iou.replace('iou', 'l1'))
        gts.append(iou.replace('iou', 'label_2'))
        images.append(iou.replace('iou', 'image_2').replace('txt', 'png'))
        calibs.append(iou.replace('iou', 'calib'))
        saves_3d_iou.append(iou.replace('iou', 'iou_3d').replace('txt', 'png'))
        saves_bev_iou.append(iou.replace('iou', 'iou_bev').replace('txt', 'png'))
        saves_3d_l1.append(iou.replace('iou', 'l1_3d').replace('txt', 'png'))
        saves_bev_l1.append(iou.replace('iou', 'l1_bev').replace('txt', 'png'))

for i in tqdm(range(len(preds_iou))):
    plot_bboxes_bev(images[i], calibs[i], preds_iou[i], gts[i], saves_bev_iou[i])
    plot_bboxes_bev(images[i], calibs[i], preds_l1[i], gts[i], saves_bev_l1[i])
    plot_bboxes_3d(images[i], calibs[i], preds_iou[i], gts[i], saves_3d_iou[i])
    plot_bboxes_3d(images[i], calibs[i], preds_l1[i], gts[i], saves_3d_l1[i])

100%|██████████| 3769/3769 [06:05<00:00, 10.30it/s]
